<br>
<br>
<center><b><font size=200>Gotta SQL 'Em All</font></b></center>
<br>

<img src='pokemon_all.jpg'></img>

## <center><b>The Database</b></center>

<img src='schema.png'></img>

## Connecting to the database

In [66]:
import sqlite3
import pandas as pd

conn = sqlite3.Connection('pokedex.db', timeout = 60)
cur = conn.cursor()

## Using SELECT

### Get a list of all Pokemon names

In [3]:
cur.execute('''
            SELECT name 
            FROM pokemon
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name
0,Bulbasaur
1,Ivysaur
2,Venusaur
3,Charmander
4,Charmeleon
...,...
146,Dratini
147,Dragonair
148,Dragonite
149,Mewtwo


## Using WHERE

### Get the names of all Pokemon that have Water as their primary type (Type1)

In [4]:
cur.execute('''
            SELECT name 
            FROM pokemon
            WHERE Type1 = 'Water'
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name
0,Squirtle
1,Wartortle
2,Blastoise
3,Psyduck
4,Golduck
5,Poliwag
6,Poliwhirl
7,Poliwrath
8,Tentacool
9,Tentacruel


## Using ORDER BY

### Get the names and weights of each Pokemon from highest weight to lowest weight

In [5]:
cur.execute('''
            SELECT name, weight
            FROM pokemon
            ORDER BY weight DESC
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,weight
0,Snorlax,4600
1,Golem,3000
2,Gyarados,2350
3,Lapras,2200
4,Onix,2100
...,...,...
146,Pidgey,18
147,Koffing,10
148,Diglett,8
149,Gastly,1


## Using LIMIT

### Get the names and heights of the top 5 tallest Pokemon

In [6]:
cur.execute('''
            SELECT name, height
            FROM pokemon
            ORDER BY height DESC
            LIMIT 5
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,height
0,Onix,88
1,Gyarados,65
2,Dragonair,40
3,Arbok,35
4,Lapras,25


## Using GROUP BY

### Get the type and average weight and height for each primary type

In [7]:
cur.execute('''
            SELECT Type1, AVG(weight) AS avg_weight, AVG(height) AS avg_height
            FROM pokemon
            GROUP BY Type1 
            ORDER BY avg_weight
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,Type1,avg_weight,avg_height
0,Ghost,135.666667,14.666667
1,Bug,229.916667,9.000000
2,Poison,273.142857,12.214286
3,Grass,279.916667,10.833333
4,Electric,317.888889,8.555556
5,Ground,452.625000,8.500000
6,Normal,478.916667,9.833333
7,Ice,480.000000,15.500000
8,Fire,480.250000,12.166667
9,Psychic,515.625000,12.500000


## Using JOIN

### Create a table of move names, type, and type ID

In [8]:
cur.execute('''
            SELECT lm.move, t.identifier, t.id
            FROM learned_moves lm
            JOIN types t
            ON lm.type_id = t.id
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,move,identifier,id
0,Swords Dance,normal,1
1,Cut,normal,1
2,Vine Whip,grass,12
3,Tackle,normal,1
4,Body Slam,normal,1
...,...,...,...
4147,Explosion,normal,1
4148,Rest,psychic,14
4149,Rock Slide,rock,6
4150,Tri Attack,normal,1


## Combining methods

### Find the two Pokemon types with the least weaknesses

In [18]:
cur.execute('''
            SELECT identifier, COUNT(attacking_type) AS num_weaknesses
            FROM weaknesses
            JOIN types
            ON defending_type = id
            WHERE damage_factor > 100
            GROUP BY identifier
            ORDER BY num_weaknesses
            LIMIT 2
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,identifier,num_weaknesses
0,electric,1
1,normal,1


### Find the top 5 Pokemon having the highest variety of move types.

In [27]:
cur.execute('''
            SELECT name, COUNT(DISTINCT type_id) AS num_vareity
            FROM pokemon
            JOIN learned_moves
            ON id = pokemon_id
            GROUP BY name
            ORDER BY num_vareity DESC 
            LIMIT 5
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,num_vareity
0,Mew,13
1,Snorlax,11
2,Kangaskhan,11
3,Rhydon,10
4,Nidoqueen,10


In [24]:
cur.execute('''
            SELECT *
            FROM learned_moves
        
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,pokemon_id,move,power,pp,level,type_id
0,1,Swords Dance,0,30,0,1
1,1,Cut,50,30,0,1
2,1,Vine Whip,35,15,13,12
3,1,Tackle,50,35,1,1
4,1,Body Slam,85,15,0,1
...,...,...,...,...,...,...
4147,151,Explosion,250,5,0,1
4148,151,Rest,0,10,0,14
4149,151,Rock Slide,75,10,0,6
4150,151,Tri Attack,80,10,0,1


### Get the names of all Pokemon who learn a super effective move against Water type Pokemon. Also include the name of one of these moves the Pokemon learns.

First, try using a subquery to get types super effective against water.

In [54]:
cur.execute('''
            SELECT identifier as type
            FROM types
            JOIN weaknesses
            ON attacking_type = id
            WHERE defending_type IN (SELECT id
                                     FROM types
                                     WHERE identifier = "water")
            AND damage_factor = 200
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


,type
0,grass
1,electric


In [64]:
cur.execute('''
            SELECT name, move
            FROM pokemon p
            JOIN learned_moves m
            ON p.id = m.pokemon_id
            JOIN types t
            ON m.type_id = t.id
            JOIN weaknesses w
            ON w.attacking_type = t.id
            WHERE w.defending_type IN (SELECT id
                                     FROM types
                                     WHERE identifier = "water")
            AND w.damage_factor = 200
            GROUP BY name
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,name,move
0,Abra,Thunder Wave
1,Alakazam,Thunder Wave
2,Arbok,Mega Drain
3,Beedrill,Mega Drain
4,Bellsprout,Mega Drain


## INSERT

### <center>A new Pokemon was discovered! </center>

<img src='celebi.png'></img>

### Add Celebi to the database. Celebi is a Psychic and Grass type Pokemon who is 2 feet tall and weighs 11 pounds. Celebi is of the genus 'onion' and has a Pokedex ID of 251.

In [91]:
cur.execute('''
            INSERT INTO Pokemon(id, name, genus, height, weight, Type1, Type2)
            VALUES(251, 'Calebi', 'Onion', 2, 11, 'Psyhic', 'Grass')
           ''')
cur.execute('''
            SELECT *
            FROM pokemon
            WHERE name = 'Calebi'
            ''')
cur.fetchall()


[(251, 'Calebi', 'Onion', 2, 11, 'Psyhic', 'Grass')]

In [90]:
cur.execute('''
            DELETE FROM pokemon WHERE name = 'Calebi';
            ''')

In [94]:
cur.execute('''
            SELECT *
            FROM pokemon
            WHERE name='Celebi'
            ''')
cur.fetchall()

[]

## Database Management

### Pokemon were recently discovered to have unique abilities. Add a new table to the database to store these abilties. Make sure to include the ability name, an ability ID, and a text description of the ability's effect.

<br>

#### Also, add the ability called Flash Fire. It should have an ID of 18 and has the effect "Immune to Fire type moves. Being attacked by a fire-type move increase the power of future Fire type attacks."

In [95]:
cur.execute('''
            CREATE TABLE Ability (
                AbilityID int,
                AbilityName varchar(225),
                Description varchar(225)
                )
            ''')

In [96]:
cur.execute('''
            INSERT INTO Ability(AbilityID, AbilityName, Description)
            VALUES(18, 'Flash Fire', 'Immune to Fire type moves. Being attacked by a fire-type move increase the power of future Fire type attacks.')
           ''')
cur.execute('''
            SELECT *
            FROM Ability
            ''')
cur.fetchall()

[(18,
  'Flash Fire',
  'Immune to Fire type moves. Being attacked by a fire-type move increase the power of future Fire type attacks.')]

## Challenge Activity

### <center>You're about to face gym leader Sabrina who has a team full of Psychic type Pokemon. 

<img src='sabrina.png'></img>

### <center>Your team currently consists of a Hitmonchan and an Arbok, both of which are weak to Psychic type attacks and have no moves that will be super effective against Psychic Pokemon.

<img src='team.jpg'></img>

### <center>Assuming the highest level of Pokemon you could currently catch is 20, you want to find a Pokemon that has an attack which is super effective against Psychic type Pokemon with the highest possible power level. That Pokemon should also not be weak to Psychic type attacks.
### <center>Which Pokemon should you try and catch to defeat Sabrina? 

#### Hints:
- Your answer should return the name of the Pokemon, the name of the move, the power of the move, and the level at which the Pokemon learns the move.
- If you need to compare text values and the columns do not have the same formatting (case-wise) you can use LOWER() to allow you to compare them.
- If you have two columns of the same name from two different tables, you have to specify which one you are going to use. For example - pokemon.id vs types.id
- A super effective move will have a damage factor greater than 100.

In [107]:
cur.execute('''
            SELECT name, move, power, level
            FROM pokemon
            JOIN learned_moves
            ON pokemon.id = pokemon_id
            JOIN types
            ON type_id = types.id
            JOIN weaknesses 
            ON attacking_type = types.id
            WHERE defending_type IN (SELECT id 
                                     FROM types 
                                     WHERE identifier = 'psychic')
            AND damage_factor = 200
            AND LOWER(Type1) IN (SELECT identifier
                                 FROM types
                                 JOIN weaknesses
                                 ON id = defending_type
                                 WHERE damage_factor <=100
                                 AND attacking_type = (SELECT id 
                                                       FROM types
                                                       WHERE identifier = 'psychic')
                                )
            AND (LOWER(Type2) = 'none' OR LOWER(Type2) IN (SELECT identifier
                                                           FROM types
                                                           JOIN weaknesses
                                                           ON id = defending_type
                                                           WHERE damage_factor <= 100
                                                           AND attacking_type = (SELECT id
                                                                                 FROM types
                                                                                 WHERE identifier = 'psyhcic')
                                                            )
                )
            AND level <= 20
            GROUP BY name
            ORDER BY power DESC, level DESC
            LIMIT 4
            ''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,move,power,level
0,Gyarados,Bite,60,20
1,Meowth,Bite,60,12
2,Growlithe,Bite,60,1
3,Persian,Bite,60,1


## Bonus

### <center>Come up with your own Pokedex question build a query to solve it.